In [2]:
from unsloth import FastLanguageModel
import torch
from tqdm.notebook import tqdm

/tmp/ipykernel_833893/4042337291.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3.2-11b-vision-instruct-bnb-4bit",
    model_name = 'Qwen/Qwen3-4B-Instruct-2507',
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen3 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

Unsloth 2025.8.5 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


In [10]:
import pandas as pd

data_df = pd.read_csv('../../data/iat_predictor_splits/clusters_df.csv')

clips_dir = '/home/firdavs/surgery/clips_with_wiggle/fb_clips_wiggle'
    
cols = ['dialogue', 'cvid', 'instrument', 'action', 'tissue', 'procedure', 'task']
data_df = data_df[cols]
data_df

,dialogue,cvid,instrument,action,tissue,procedure,task
0,"check on the left side, just see what's uhh.. ...",c1_s0_0-35-52.avi,NONE,NONE,NONE,radical prostatectomy,Bladder neck
1,just give that a little tap there,c1_s0_0-36-30.avi,NONE,NONE,NONE,radical prostatectomy,Bladder neck
2,alright that's fine,c1_s0_0-36-32.avi,NONE,NONE,NONE,radical prostatectomy,Bladder neck
3,alright let's do a little uh... uh... mhm.. th...,c1_s0_0-36-34.avi,NONE,NONE,prostate_tissue,radical prostatectomy,Bladder neck
4,yeah,c1_s0_0-36-41.avi,NONE,NONE,NONE,radical prostatectomy,Bladder neck
...,...,...,...,...,...,...,...
4205,on the sides,c33_s0_1-24-51.avi,NONE,NONE,NONE,radical prostatectomy,bladder neck
4206,uh huh,c33_s0_1-24-57.avi,NONE,NONE,NONE,radical prostatectomy,bladder neck
4207,on the other side,c33_s0_1-25-1.avi,NONE,NONE,NONE,radical prostatectomy,bladder neck
4208,"too far, come a little bit closer to the prostate",c33_s0_1-25-5.avi,NONE,NONE,prostate_tissue,radical prostatectomy,bladder neck


In [11]:
from typing import Tuple
import cv2
import numpy as np

def load_and_sample_video(clip_path: str, resize_hw: Tuple[int, int], num_frames: int = None) -> np.ndarray:
    """Loads, samples, and resizes video frames."""
    cap = cv2.VideoCapture(clip_path)
    if not cap.isOpened():
        raise RuntimeError(f"Error opening video: {clip_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        cap.release()
        raise ValueError("Video is empty")

    # Uniformly sample indices
    if num_frames is None:
        num_frames = total_frames
        
    if total_frames <= num_frames:
        indices = list(range(total_frames)) + [total_frames - 1] * (num_frames - total_frames)
    else:
        step = total_frames / float(num_frames)
        indices = [min(int(i * step), total_frames - 1) for i in range(num_frames)]
    
    frames = []
    for idx in sorted(list(set(indices))): # Read each unique frame only once
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ok, frame = cap.read()
        if ok:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            H, W = resize_hw
            frame = cv2.resize(frame, (W, H), interpolation=cv2.INTER_AREA)
            frames.append(frame)
    cap.release()

    # Create the final list by selecting from the read frames
    frame_map = {idx: frame for idx, frame in zip(sorted(list(set(indices))), frames)}
    sampled = [frame_map[i] for i in indices]

    return np.stack(sampled, axis=0) # [T,H,W,C]


cvid = 'c1_s0_1-11-23.avi'

frames = load_and_sample_video(
    clip_path = f"{clips_dir}/{cvid}",
    resize_hw = (224, 224),
)
num_frames = frames.shape[0]
frames.shape

(50, 224, 224, 3)

In [12]:
import torchvision.transforms as T
torch2PIL = T.ToPILImage()

instruction = 'Respond with 1-2 sentences of actionable feedback focused on the most teachable instrument-action-tissue event.'

question_template = "Procedure: {procedure}\nTask: {task}"

sample_frames = 10

def formatting_prompts_func(row):
    procedure = row["procedure"]
    task = row["task"]
    answer = row["dialogue"]
    cvid = row["cvid"]
    
    try:
        video = load_and_sample_video(
            clip_path = f"{clips_dir}/{cvid}",
            resize_hw = (224, 224),
            num_frames = sample_frames
        )
    except Exception as e:
        print(f"Error loading video {cvid}: {e}")
        return {'image': [None] * sample_frames, 'text': [None] * sample_frames}
    
    question = question_template.format(procedure=procedure, task=task)
    
    texts = []
    images = []
    for i in range(sample_frames):
        image = video[i]
        images.append(image)
        text = [
            {"role": "user", "content": [
                {"type": "text", "text": instruction},
                {"type": "image"}, # Placeholder for the image
                {"type": "text", "text": question},
                ]
            },
            {"role": "assistant", "content": [
                {"type": "text", "text": answer},
                ]
            },
        ]
        text = tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=False)
        texts.append(text)
        
    return { "text" : texts, "image" : images, }

In [16]:
from datasets import Dataset
from joblib import Parallel, delayed
from typing import Dict, List

train_df = data_df.sample(frac=0.8, random_state=42).reset_index(drop=True)
val_df = data_df.drop(train_df.index).reset_index(drop=True)

def process_batch(batch: pd.DataFrame) -> List[Dict]:
    rows = []
    for i in range(len(batch)):
        row = batch.iloc[i]
        result = formatting_prompts_func(row)
        images = result['image']
        texts = result['text']
        for j in range(sample_frames):
            new_row = row.to_dict()
            new_row['i'] = i
            new_row['frame_idx'] = j
            new_row['image'] = images[j]
            new_row['text'] = texts[j]
            rows.append(new_row)
    return rows
    
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
    
n_jobs = 12
batch_size = 50
train_rows = Parallel(n_jobs=n_jobs, backend='multiprocessing')(
    delayed(process_batch)(train_df[i:i+batch_size]) for i in tqdm(range(0, len(train_df), batch_size), desc="Processing train batches")
)

val_rows = Parallel(n_jobs=n_jobs, backend='multiprocessing')(
    delayed(process_batch)(val_df[i:i+batch_size]) for i in tqdm(range(0, len(val_df), batch_size), desc="Processing validation batches")
)

Processing train batches:   0%|          | 0/68 [00:00<?, ?it/s]

Error loading video c20_s0_2-22-56.avi: Error opening video: /home/firdavs/surgery/clips_with_wiggle/fb_clips_wiggle/c20_s0_2-22-56.avi


Processing validation batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [17]:
if isinstance(train_rows[0], list):
    train_rows = [item for sublist in train_rows for item in sublist]
if isinstance(val_rows[0], list):
    val_rows = [item for sublist in val_rows for item in sublist]

In [18]:
train_df = pd.DataFrame(train_rows).rename(columns={'frame_idx': 'second', 'dialogue': 'answer'}).dropna(subset=['image', 'text'])
val_df = pd.DataFrame(val_rows).rename(columns={'frame_idx': 'second', 'dialogue': 'answer'}).dropna(subset=['image', 'text'])

print(train_df.shape, val_df.shape)

(33670, 11) (8420, 11)


In [19]:
train_df.iloc[0]

answer                                      yeah lets go for it
cvid                                          c9_s0_0-11-28.avi
instrument                                                 NONE
action                                                     NONE
tissue                                                     NONE
procedure                                  simple prostatectomy
task                                                  Cystotomy
i                                                             0
second                                                        0
image         [[[8, 6, 11], [41, 21, 18], [63, 31, 20], [63,...
text          <|im_start|>user\n<|im_end|>\n<|im_start|>assi...
Name: 0, dtype: object

In [20]:
from datasets import Features, Value, Image

features = Features({
    'answer': Value('string'),
    'cvid': Value('string'),
    'instrument': Value('string'),
    'action': Value('string'),
    'tissue': Value('string'),
    'procedure': Value('string'),
    'task': Value('string'),
    'i': Value('int64'),
    'second': Value('int64'),
    'text': Value('string'),
    
    'image': Image() 
})

train_data_dict = train_df.to_dict('list')
val_data_dict = val_df.to_dict('list')

In [21]:
train_ds = Dataset.from_dict(train_data_dict, features=features)

In [22]:
val_ds = Dataset.from_dict(val_data_dict, features=features)

In [23]:
train_ds

Dataset({
    features: ['answer', 'cvid', 'instrument', 'action', 'tissue', 'procedure', 'task', 'i', 'second', 'image', 'text'],
    num_rows: 33670
})

In [24]:
val_ds

Dataset({
    features: ['answer', 'cvid', 'instrument', 'action', 'tissue', 'procedure', 'task', 'i', 'second', 'image', 'text'],
    num_rows: 8420
})

In [ ]:
# save datasets
train_ds.save_to_disk("../../outputs/vllm_datasets/train_ds")
val_ds.save_to_disk("../../outputs/vllm_datasets/val_ds")

Saving the dataset (0/7 shards):   0%|          | 0/33670 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/8420 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

print("Loading train...")
train_ds = load_dataset("../../outputs/vllm_datasets/train_ds")
print("Loading val dataset...")
val_ds = load_dataset("../../outputs/vllm_datasets/val_ds")

Loading train...


FileNotFoundError: Couldn't find any data file at /home/firdavs/surgery/surgical_fb_generation/SurgFBGen/outputs/llama-3.2-11b-vqav2-surgical-cholecT50---10frames_proc+task/train_ds.

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = val_ds,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training faster for many short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = -1, # Set to a higher value for a full finetune
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "../../outputs/qwen3-4b-instruct",
        report_to = "none",
    ),
)

trainer.train()

Unsloth: Tokenizing ["text"]:   0%|          | 0/33670 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/8420 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 33,670 | Num Epochs = 3 | Total steps = 12,627
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 33,030,144 of 4,055,498,240 (0.81% trained)


TorchRuntimeError: Dynamo failed to run FX node with fake tensors: call_function <built-in function mul>(*(FakeTensor(..., device='cuda:0', size=(s85, s35, s0, s34),
           dtype=torch.bfloat16), FakeTensor(..., device='cuda:0', size=(s56, 1, s34), dtype=torch.bfloat16)), **{}): got RuntimeError('The size of tensor a (s35) must match the size of tensor b (s56) at non-singleton dimension 1)')

from user code:
   File "/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/surgfbgen/notebooks/unsloth_compiled_cache/unsloth_compiled_module_qwen3.py", line 242, in apply_rotary_pos_emb
    q_embed = (q * cos) + (rotate_half(q) * sin)

Set TORCHDYNAMO_VERBOSE=1 for the internal stack trace (please do this especially if you're reporting a bug to PyTorch). For even more developer context, set TORCH_LOGS="+dynamo"


In [ ]:
model.save_pretrained("../../outputs/qwen3-4b-instruct/lora_model")